In [1]:
# GaChalign (Length-based)
# https://github.com/alvations/gachalign => futurize

import bucc_proc as bp
import matplotlib.pyplot as plt
from random import randrange
from pylab import polyfit
import importlib
importlib.reload(bp)

from gachalign import length_cost

pd = bp.pd
Path = bp.Path

#length_cost(sx, sy, mean_xy, variance_xy)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
new_df = bp.get_merge()
new_df.count

Merged file exists, reading...


<bound method DataFrame.count of              ID_zh         ID_en                             Sentence_zh  \
0     zh-000000033  en-000005983               1989年以前，全球经济包含大约8亿到10亿人口。   
1     zh-000000231  en-000047360        今日全球面临的威胁是超民族的，因此也必须采取超民族的方式来应对。   
2     zh-000000272  en-000027140                   欧盟移民政策的硬伤还有一个不太显著的方面。   
3     zh-000000438  en-000065621           只有让民粹主义服务于自由主义改革，政府才能取得长久的利益。   
4     zh-000000639  en-000005169       但社会民主派必须理解为何示威的发展会独立于现有的有组织中左翼政治。   
...            ...           ...                                     ...   
1848  zh-000094590  en-000013258         事件发生后当局在尚未进行调查的情况下就匆匆掩埋了出事列车残骸。   
1849  zh-000094593  en-000061419             北方拥有丰富的自然资源，就连电力也是从北方输送到南方。   
1850  zh-000094607  en-000039373                如果利率为3%，那么年税收额必须增加15亿美元。   
1851  zh-000094611  en-000003807           五年前，叙利亚北部边陲城镇享受着土耳其高速经济增长的红利。   
1852  zh-000094633  en-000083972  在过去的一个世纪中，我们的世界发生了翻天覆地的变化——技术是其中的重要原因。   

                                            Sentence_e

In [3]:
def get_length(sentence, groupby="char", lang ="en"):
    if groupby == "char":
        n = len(sentence)
        
    elif groupby == "word":
        if lang == "en":
            n = len(bp.word_tokenize(sentence))
        elif lang == "zh":
            n = len(bp.jieba.lcut(sentence))
            
    elif groupby == "special":
        # count each Chinese character as having length 2, and each English or punctuation character as having length 1
        if lang == "zh":
            n=0
            for c in sentence:
                if c in bp.punctuation:
                    n+=1
                else:
                    n+=2
        else:
            n = len(sentence)
            
    return n

In [4]:
gc_file = Path("zh-en.training.gc")

if gc_file.is_file():
    print('GC file exists, reading...')
    gc_df = pd.read_csv(gc_file, header=0, sep='\t')

else:
    
    print('GC file does not exist, creating...')
    
    gc_df = new_df
    gc_df['char_en'] = [get_length(new_df.iloc[i]['Sentence_en']) for i in range(1853)]
    gc_df['char_zh'] = [get_length(new_df.iloc[i]['Sentence_zh'], lang="zh") for i in range(1853)]
    gc_df['word_en'] = [get_length(new_df.iloc[i]['Sentence_en'], "word") for i in range(1853)]
    gc_df['word_zh'] = [get_length(new_df.iloc[i]['Sentence_zh'], "word", "zh") for i in range(1853)]
    gc_df['special_en'] = [get_length(new_df.iloc[i]['Sentence_en'], "special") for i in range(1853)]
    gc_df['special_zh'] = [get_length(new_df.iloc[i]['Sentence_zh'], "special", "zh") for i in range(1853)]
    gc_df.to_csv(gc_file, index=False, sep='\t')

GC file exists, reading...


In [14]:
def create_negatest(t,n=0, groupby="special"):
    string = ''
    
    while t > n:
        x = randrange(1853)
        y = randrange(1853)
        if x != y:
            en_len = get_length(new_df.iloc[x]['Sentence_en'], groupby=groupby)
            zh_len = get_length(new_df.iloc[y]['Sentence_zh'], lang="zh", groupby=groupby)
            
            x = str(x)
            y = str(y)
            en_len = str(en_len)
            zh_len = str(zh_len)
            
            string = string + x + '\t' + y + '\t' + en_len + '\t' + zh_len + '\n'
            n+=1
    
    
    string = 'Line_en\tLine_zh\tlength_en\tlength_zh\n' + string
    return string
            
def str_to_txt(filename, string):
    with open(filename,'w', encoding='utf-8') as file:
        file.write(string)


inc_file = Path("zh-en.training.incorrect")
if inc_file.is_file():
    print('INC File exists, reading...')
    inc_df = pd.read_csv(inc_file, header=0, sep='\t')
else:
    print('GC file does not exist, creating...')
    str_to_txt('zh-en.training.incorrect',create_negatest(1853))

inc_df

INC File exists, reading...


,Line_en,Line_zh,length_en,length_zh
0,647,430,126,52
1,1835,1362,56,69
2,365,15,80,64
3,1373,701,89,43
4,197,987,114,60
...,...,...,...,...
1848,1308,962,93,64
1849,801,602,108,34
1850,1778,1359,91,58
1851,1782,1097,78,72


In [13]:
c_df = pd.DataFrame()
c_df['Line_en'] = gc_df.index
c_df['Line_zh'] = gc_df.index
c_df['length_en'] = gc_df['special_en']
c_df['length_zh'] = gc_df['special_zh']
c_df

,Line_en,Line_zh,length_en,length_zh
0,0,0,85,48
1,1,1,103,62
2,2,2,82,41
3,3,3,102,56
4,4,4,129,65
...,...,...,...,...
1848,1848,1848,93,61
1849,1849,1849,100,52
1850,1850,1850,87,45
1851,1851,1851,106,56


In [51]:
def _f_score(x, cut_point, c=1, s2=6.8):
    p,n = (0,0)
    
    if length_cost([x['length_zh']], [x['length_en']], c, s2) < cut_point:
        p += 1
    else:
        n +=1
    return p, n

def f_score(cut_point, c=1, s2=6.8):
    
    tp = c_df.apply(lambda x: _f_score(x, cut_point, c, s2), axis=1).value_counts().get((1,0))
    fn = c_df.apply(lambda x: _f_score(x, cut_point, c, s2), axis=1).value_counts().get((0,1))
    fp = inc_df.apply(lambda x: _f_score(x, cut_point, c, s2), axis=1).value_counts().get((1,0))
    tn = inc_df.apply(lambda x: _f_score(x, cut_point, c, s2), axis=1).value_counts().get((0,1))
    
    tp = tp if tp else 0
    fn = fn if fn else 0
    fp = fp if fp else 0
    tn = tn if tn else 0
    
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1 = 2*(precision*recall/(precision+recall))
    
    return f1*100
    
    
start_c = 518.5
step_c = 0.125
num_c = 20

f_df = pd.DataFrame(columns=['Criteria','F1_Score(%)'])

n=0 
while n<num_c:
    criteria = start_c
    
    f_df.loc[len(f_df)] = [criteria, f_score(criteria)]
    
    start_c += step_c
    n+=1
    
f_df

,Criteria,F1_Score(%)
0,518.500,68.016654
1,518.625,68.016654
2,518.750,68.016654
3,518.875,68.003784
4,519.000,68.003784
5,519.125,68.003784
6,519.250,68.003784
7,519.375,68.003784
8,519.500,68.003784
9,519.625,67.990919


In [53]:
# From above, best cut-off is 519 with highest F1 score of 68%
# Best achievable F1 score from this dataset is 68%
# cut-off will differ per data, let machine learning decide

In [75]:
# Source is ZH
# Target is EN

## Calculate Mean from dataset

print(c_df['length_en'].sum()/c_df['length_zh'].sum())


## Calculate Variance from dataset
diffsquares = [(c_df['length_en'][i]-c_df['length_zh'][i])**2 for i in range(1853)]
src_paragraph_len = [c_df['length_zh'][i] for i in range(1853)]
(m,_)= polyfit(src_paragraph_len,diffsquares,1)
print(m)

1.734477180253934
5.337059468806532


In [81]:
# Investigate optimum c and s2
# Gachalign for japanese is (1.332, 77.64) but this was derived from their word count method
# which differs from 1 ZH char is 2 special counting
# https://drive.google.com/file/d/1XxCLxA99MRJbF6xzWS8WDR9igrz-m2q8/view

c = 1.734477180253934

start_c = 1600
step_c = 1
num_c = 20

f_df = pd.DataFrame(columns=['Criteria','F1_Score(%)'])

n=0 
while n<num_c:
    criteria = start_c
    
    f_df.loc[len(f_df)] = [criteria, f_score(criteria, c)]
    
    start_c += step_c
    n+=1
    
f_df

,Criteria,F1_Score(%)
0,1600.0,67.462687
1,1601.0,67.462687
2,1602.0,67.487409
3,1603.0,67.487409
4,1604.0,67.499534
5,1605.0,67.548928
6,1606.0,67.536340
7,1607.0,67.536340
8,1608.0,67.523756
9,1609.0,67.498603


In [86]:
# Investigate s2

c = 1.734477180253934
s2 = 5.337059468806532

start_c = 2000
step_c = 1
num_c = 10

f_df = pd.DataFrame(columns=['Criteria','F1_Score(%)'])

n=0 
while n<num_c:
    criteria = start_c
    
    f_df.loc[len(f_df)] = [criteria, f_score(criteria, c, s2)]
    
    start_c += step_c
    n+=1
    
f_df

,Criteria,F1_Score(%)
0,2000.0,67.487409
1,2001.0,67.487409
2,2002.0,67.499534
3,2003.0,67.548928
4,2004.0,67.548928
5,2005.0,67.536340
6,2006.0,67.536340
7,2007.0,67.536340
8,2008.0,67.498603
9,2009.0,67.498603


In [89]:
# Findings: Varying Mean and Variance does not imporve highest F1 score achievable